In [2]:
import numpy as np
from numpy import zeros, ones, expand_dims
from numpy.random import randn, randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D,\
Conv2DTranspose, LeakyReLU, BatchNormalization, Dropout, Embedding, \
Activation, Concatenate
from keras.initializers import RandomNormal
import matplotlib.pyplot as plt

In [3]:
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=in_shape)
    dis = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    dis = LeakyReLU(alpha=0.2)(dis)
    dis = Dropout(0.5)(dis)
    dis = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(dis)
    dis = BatchNormalization()(dis)
    dis = LeakyReLU(alpha=0.2)(dis)
    dis = Dropout(0.5)(dis)
    dis = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(dis)
    dis = BatchNormalization()(dis)
    dis = LeakyReLU(alpha=0.2)(dis)
    dis = Dropout(0.5)(dis)
    dis = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(dis)
    dis = BatchNormalization()(dis)
    dis = LeakyReLU(alpha=0.2)(dis)
    dis = Dropout(0.5)(dis)
    dis = Flatten()(dis)
    out1 = Dense(1, activation='sigmoid')(dis)
    out2 = Dense(n_classes, activation='softmax')(dis)
    model = Model(in_image, [out1, out2])
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss = ['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [4]:
def define_generator(latent_dim, n_classes=10):
    init = RandomNormal(stddev=0.2)
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(7*7, kernel_initializer=init)(li)
    li = Reshape((7,7,1))(li)
    in_lat = Input(shape=(latent_dim, ))
    gen = Dense(384*7*7, kernel_initializer=init)(in_lat)
    gen = Activation('relu')(gen)
    gen = Reshape((7,7,384))(gen)
    merge = Concatenate()([gen, li])
    gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
    gen = BatchNormalization()(gen)
    gen = Activation('relu')(gen)
    gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    out_layer = Activation('tanh')(gen)
    model = Model([in_lat, in_label], out_layer)
    return model

In [5]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    gan_output = d_model(g_model.output)
    model = Model(g_model.input, gan_output)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [6]:
def load_real_samples():
    (trainX, trainy), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = X - 127.5 / 127.5
    return [X, trainy]

In [7]:
def generate_real_samples(dataset, n_samples):
    images, labels = dataset
    idx = randint(0, images.shape[0], n_samples)
    X, labels = images[idx], labels[idx]
    y = ones((n_samples, 1))
    return [X, labels], y

In [8]:
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape((n_samples, latent_dim))
    labels = randint(0, n_classes, n_samples)
    return [x_input, labels]

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input, labels = generate_latent_points(latent_dim, n_samples)
    images = generator.predict([x_input, labels])
    y = zeros((n_samples, 1))
    return [images, labels], y

In [17]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    [X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = X + 1 / 2.0
    for i in range(100):
        plt.subplot(10,10,i+1)
        plt.axis('off')
        plt.imshow(X[i, :, :, 0], cmap='gray_r')
    filename1 = 'generated_plot_%04d.png' %(step+1)
    plt.savefig(filename1)
    plt.close()
    filename2 = 'model_%04d.h5' %(step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [13]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=64):
    batch_per_epoch = int(dataset[0].shape[0] / n_batch)
    n_steps = batch_per_epoch * n_epochs
    half_batch = int(n_batch / 2)
    for i in range(n_steps):
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        _, d_r1, d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        _, d_f, d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
        y_gan = ones((n_batch, 1))
        _, g_1, g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        print(">%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]" % (i+1, d_r1, d_r2, d_f, d_f2, g_1, g_2))
        if (i+1) % (batch_per_epoch * 10) == 0:
            summarize_performance(i, g_model, latent_dim)
            

In [14]:
latent_dim = 100
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)
dataset = load_real_samples()

In [18]:
train(generator, discriminator, gan_model, dataset, latent_dim)

In [20]:
#genearting desired class of data 
from math import sqrt
from numpy import asarray

In [21]:
def generate_latent_points(latent_dim, n_samples, n_class):
    x_input = randn(latent_dim * n_samples)
    z_input = x_input.reshape((n_samples, latent_dim))
    labels = asarray([n_class for _ in range(n_samples)])
    return [z_input, labels]

In [25]:
def save_plot(examples, n_examples):
    for i in range(n_examples):
        plt.subplot(10, 10, 1+i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    plt.show()

In [26]:
model = load_model('model_93700.h5')
latent_dim = 100
n_examples = 100
n_class = 7
latent_points, labels = generate_latent_points(latent_dim, n_examples, n_class)
X = model.predict([latent_points, labels])
X = X + 1 / 2.0
save_plot(X, n_examples)